In [3]:
pip install sentence-transformers 

  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 7.1/7.1 MB 18.2 MB/s eta 0:00:00
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached torch-2.0.1-cp311-cp311-win_amd64.whl (172.3 MB)
  Using cached torchvision-0.15.2-cp311-cp311-win_amd64.whl (1.2 MB)
  Using cached scikit_learn-1.2.2-cp311-cp311-win_amd64.whl (8.3 MB)
  Using cached scipy-1.10.1-cp311-cp311-win_amd64.whl (42.2 MB)
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached sentencepiece-0.1.99-cp311-cp311-win_amd64.whl (977 kB)
  Using cached huggingface_hub-0.14.1-py3-none-any.whl (224 kB)
  Using cached filelock-3.12.0-py3-none-any.whl (10 kB)
  Using cached fsspec-2023.5.0-py3-none-any.whl (160 kB)
  Using cached PyYAML-6.0-cp311-cp311-win_amd64.whl (143 kB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached networkx-3.1-py

  DEPRECATION: sentence-transformers is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer  # 한국어 처리 모델
from sklearn.metrics.pairwise import cosine_similarity  # 유사도 분석

c:\Users\Playdata\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 데이터셋 로드

원천 데이터 (.txt)

In [ ]:
'''
import zipfile
# 추출할 zip 파일 경로
for i in range(1,5):
    zip_file_path = f"C:/dev/github/mental-health-chatbot/data/Training/TS_01. KAKAO({i}).zip"
    # 추출할 위치
    extract_path = "C:/dev/github/mental-health-chatbot/data/Training/data"
    # zipfile 객체 생성
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # 추출
        zip_ref.extractall(extract_path)
'''

In [6]:
# 읽어올 파일들이 들어있는 폴더 경로
folder_path = "C:/dev/github/mental-health-chatbot/data/Training/data"

# 폴더 내의 모든 txt 파일 경로 리스트 생성
file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.txt')]

# 각 파일을 읽어와서 DataFrame으로 합치기
i = 0
df_list = []
for file_path in file_paths:
    with open(file_path, 'r', encoding = "utf-8") as f:
        i += 1

        # 각 파일의 내용을 DataFrame으로 변환
        df = pd.read_csv(f, sep='\t', header=None)
        df['구분'] = i 
        df_list.append(df)

# 모든 DataFrame 합치기
result_df = pd.concat(df_list, ignore_index= False)

KeyboardInterrupt: 

In [ ]:
result_df

,0,구분
0,1 : 애덜앙 나 너무 배불러서 배 아파,1
1,2 : 배 아프면 약국 가서 소화제라도 마시는 거 어때,1
2,3 : 얼마나 먹었다고 배가 아프데,1
3,1 : 떡볶이랑 튀김이랑 닭껍질튀김이랑 마카롱 먹었어,1
4,2 : 하긴 *이 배는 파리보다 작으니까 당연히 그만큼 먹어도 배부르지,1
...,...,...
11,2 : 그러니깐 ㅠ 이제 해리 자식들로 시작해야지,71651
12,1 : 근데 해리만큼 잘 할 수 있을까?,71651
13,2 : 진짜 그 3인방 케미가 최고였는데,71651
14,1 : 그치그치 완전 최고였는데 ㅜㅠ,71651


라벨링 데이터 (.json)

In [ ]:
'''
# 추출할 zip 파일 경로
for i in range(1,5):
    zip_file_path = f"C:/dev/github/mental-health-chatbot/data/Training/TL_01. KAKAO({i}).zip"

    # 추출할 위치
    extract_path = "C:/dev/github/mental-health-chatbot/data/Training/data_labeling"

    # zipfile 객체 생성
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # 추출
        zip_ref.extractall(extract_path)
'''

In [ ]:
'''
import zipfile

# 추출할 zip 파일 경로
zip_file_path = "C:/dev/github/mental-health-chatbot/data/Validation/VL_01. KAKAO.zip"

# 추출할 위치
extract_path = "C:/dev/github/mental-health-chatbot/data/Validation/VL_01. KAKAO"

# zipfile 객체 생성
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:

    # 추출
    zip_ref.extractall(extract_path)
'''

In [ ]:
# 읽어올 파일들이 들어있는 폴더 경로
folder_path = "C:/dev/github/mental-health-chatbot/data/Training/data_labeling"

# 폴더 내의 모든 txt 파일 경로 리스트 생성
file_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.json')]

# 각 파일을 읽어와서 DataFrame으로 합치기
i = 0
df_list = []
for file_path in file_paths:
    with open(file_path, 'r', encoding = "utf-8") as f:
        i += 1

        # 각 파일의 내용을 DataFrame으로 변환
        df = pd.read_csv(f, sep='\t', header=None)
        df['구분'] = i 
        df_list.append(df)

# 모든 DataFrame 합치기
result_df_json = pd.concat(df_list, ignore_index= False)

In [ ]:
result_df_json

,0,구분
0,{,1
1,"""dataset"": {",1
2,"""identifier"": 71459,",1
3,"""name"": ""KAKAO_1000_01_set"",",1
4,"""src_path"": ""/data/file/cubeManager/PROJEC...",1
...,...,...
222,],71651
223,},71651
224,},71651
225,],71651


In [ ]:
result_df_json[0][24]

24                 "subject": "식음료",
24              "subject": "주거와 생활",
24                  "subject": "교통",
24            "subject": "회사/아르바이트",
24                  "subject": "군대",
                   ...              
24                  "subject": "미용",
24                  "subject": "건강",
24              "subject": "상거래 전반",
24               "subject": "방송/연예",
24               "subject": "영화/만화",
Name: 0, Length: 71651, dtype: object

In [ ]:
result_df_json[0][24].values

array(['        "subject": "식음료",', '        "subject": "주거와 생활",',
       '        "subject": "교통",', ..., '        "subject": "상거래 전반",',
       '        "subject": "방송/연예",', '        "subject": "영화/만화",'],
      dtype=object)

# 전처리

In [ ]:
result_df

,0,구분
0,1 : 애덜앙 나 너무 배불러서 배 아파,1
1,2 : 배 아프면 약국 가서 소화제라도 마시는 거 어때,1
2,3 : 얼마나 먹었다고 배가 아프데,1
3,1 : 떡볶이랑 튀김이랑 닭껍질튀김이랑 마카롱 먹었어,1
4,2 : 하긴 *이 배는 파리보다 작으니까 당연히 그만큼 먹어도 배부르지,1
...,...,...
11,2 : 그러니깐 ㅠ 이제 해리 자식들로 시작해야지,71651
12,1 : 근데 해리만큼 잘 할 수 있을까?,71651
13,2 : 진짜 그 3인방 케미가 최고였는데,71651
14,1 : 그치그치 완전 최고였는데 ㅜㅠ,71651


In [ ]:
# state = result_df_json[0][24].str.split().str[-1]

# new_state = [i : v  for i, v enumerate state]

# result_df.구분 = new_state를 ndarray로 변경해서 result_df 구분 컬럼에 삽입

In [ ]:
result_df_json[0][24]

24                 "subject": "식음료",
24              "subject": "주거와 생활",
24                  "subject": "교통",
24            "subject": "회사/아르바이트",
24                  "subject": "군대",
                   ...              
24                  "subject": "미용",
24                  "subject": "건강",
24              "subject": "상거래 전반",
24               "subject": "방송/연예",
24               "subject": "영화/만화",
Name: 0, Length: 71651, dtype: object

In [ ]:
for group, data in result_df.groupby('구분'):
    s = result_df_json[0][24]
    result_df.extend(s)

result_df['subject'] = result_df

AttributeError: 'DataFrame' object has no attribute 'extend'